# Zadanie nr 6
**Termin: 28 maja 2021 23:59**

Instrukcje

Wzorce dwuwymiarowe:
   1. Zaimplementuj algorytm wyszukiwania wzorca 2-wymiarowego
   2. Znajdź w załączonym pliku "haystack.txt" wszyskie sytuacje, gdy taka sama litera występuje na tej samej pozycji w dwóch kolejnych linijkach. Zwróć uwagę, na nierówną długość linii w pliku.
   3. Znajdź wszystkie wystąpienia "th" oraz "t h" w dwóch kolejnych liniach na tej samej pozycji.
   4. Wybierz przynajmniej 3 litery (małe). Znajdź wszystkie wystąpienia tej litery w załączonym pliku "haystack.png"
   5. Znajdź wszystkie wystąpienia słowa "p a t t e r n" w haystack.png.
   6. Porównaj czas budowania automatu i czas wyszukiwania dla różnych rozmiarów wzorca
   7. Podziel plik na 2, 4 i 8 fragmentów (w poziomie) i porównaj czas przeszukiwania

    Załączone wzorce to fragmenty książki "Jewels of Stringology".

**Wczytywanie obrazów**

In [1]:
from PIL import Image
import numpy as np

In [2]:
def load_image(image_path):
    im = Image.open(image_path)
    np_im = np.array(im)
    arr_2d = np_im[:,:, 0]
    return np.where(arr_2d > 100, 1, 0)

**Wczytanie linii z  pliku**

In [3]:
with open("haystack.txt", "r", encoding="UTF-8") as file:
    lines = file.readlines()
    lines = list(map(lambda line: line.lower(), lines))


## Zad 1

**Algorytm tworzenia i przeszukiwania Aho-Corasick**

In [4]:
from queue import Queue


class Node:
    def __init__(self, parent=None, letter=None, fail_node=None, state_num=0):
        self.parent = parent
        self.letter = letter  # letter by which the parent connects to the child
        self.children = dict()
        self.fail_node = fail_node
        self.state_num = state_num  # state_num > 0 indicate accepting state


class AhoCorasickAutomata:
    def __init__(self, patterns_list):
        self.root = Node()
        for curr_suffix in patterns_list:
            head, index = self.find(curr_suffix)
            self.graft(head, curr_suffix[index:])

        # To this point trie is built, we need to make fail_nodes now
        self.add_state_nums(patterns_list)
        self.add_fail_nodes()

    def find(self, text):
        curr_node = self.root
        i = 0
        while i < len(text) and text[i] in curr_node.children:
            curr_node = curr_node.children[text[i]]
            i += 1
        return curr_node, i

    def graft(self, node, text):
        for ch in text:
            new_node = Node(node, ch, self.root)  # creating new node with node as a parent
            node.children[ch] = new_node  # adding new node to children of current node
            node = new_node

    def search(self, substring):  # we don't count the $ sign at the end
        found_node, index = self.find(substring)
        return found_node.children == {} and index == len(substring)

    def add_fail_nodes(self):
        queue = Queue()
        queue.put(self.root)

        while not queue.empty():
            node = queue.get()
            if node != self.root and node.parent != self.root:
                parent_node, letter = node.parent, node.letter
                curr_fail = parent_node.fail_node
                while curr_fail != parent_node and letter not in parent_node.children:
                    curr_fail = curr_fail.fail_node
                if letter in curr_fail.children:
                    node.fail_node = curr_fail.children[letter]

            for letter, child_node in node.children.items():
                queue.put(child_node)

    def add_state_nums(self, list_of_patterns):
        acc, already_seen = 0, set()
        patterns = list(map(lambda arr: tuple(arr), list_of_patterns))
        for pattern in patterns:
            if pattern not in already_seen:
                node = self.root
                for character in pattern:
                    node = node.children[character]
                acc += 1
                node.state_num = acc
                already_seen.add(pattern)

    def traverse_line(self, elem_list):  # elem_list: int list, returns int list
        state = self.root
        result = []
        for elem in elem_list:
            while state is not None and elem not in state.children:
                state = state.fail_node
            if state is None:
                state = self.root
            else:
                state = state.children[elem]
            result.append(state.state_num)
        return result

    def print_aho_corasick(self, node=None):
        if node is None:  node = self.root
        print(f"Curr node={node.state_num}")
        print(f"Fail_node={node.fail_node.state_num}")
        for child in node.children.values():
            self.print_aho_corasick(child)

**Algorytm KMP**

In [5]:
def compute_pi(pattern):
    m = len(pattern)
    pi = [0 for _ in range(m)]
    k = 0
    for q in range(1, m):
        while k > 0 and pattern[k] != pattern[q]:
            k = pi[k-1]
        if pattern[k] == pattern[q]:
            k += 1
        pi[q] = k
    return pi

def pattern_matching_KMP(word, pattern, pi = None):
    result = []
    n, m, q = len(word), len(pattern), 0
    if pi is None: pi = compute_pi(pattern)
        
    for i in range(n):
        while q > 0 and pattern[q] != word[i]:
            q = pi[q-1]
        if pattern[q] == word[i]:
            q += 1
        if q == m:
            result.append(i - q + 1)
            q = pi[q-1]
    return result

**Wyszukiwanie wzorców 2-d**

In [6]:
def find_2d_pattern(elem_2d_list, patterns_list, aho_corasick=None):
    def pattern_to_int_list():
        acc, already_seen = 0, dict()
        result = []
        patterns = list(map(lambda arr: tuple(arr), patterns_list))
        for pattern in patterns:
            if pattern not in already_seen:
                acc += 1
                already_seen[pattern] = acc
            result.append(already_seen[pattern])
        return result

    def divide_column_into_words(no_col, low_threshold, n):
        result = {}
        i = 0
        while i < n:
            start = i
            new_word = []
            while i < n and no_col < len(matched_patterns_array[i]):
                new_word.append(matched_patterns_array[i][no_col])
                i += 1
            if len(new_word) >= low_threshold:
                result[start] = new_word
            i += 1
        return result

    n, m = len(elem_2d_list), len(patterns_list)
    pattern_len = len(patterns_list[0])
    aho_corasick = aho_corasick if aho_corasick is not None else AhoCorasickAutomata(patterns_list) 
    matched_patterns_array = [aho_corasick.traverse_line(line) for line in elem_2d_list]
    pattern_to_find = pattern_to_int_list()
    max_line_len = max([len(line) for line in elem_2d_list])
    

    result = []
    for i in range(max_line_len):
        words_in_column = divide_column_into_words(i, m, n)
        for row_num, word in words_in_column.items():
            matched_2d_patterns = pattern_matching_KMP(word, pattern_to_find)
            for relative_row_num in matched_2d_patterns:
                pattern_position = (row_num + relative_row_num, i - pattern_len+1)
                result.append(pattern_position)
    return result
            
    
    
test_table = ["this is a test",
              "this is fun",
              "this is fine"]
find_2d_pattern(test_table, ["this", "this", "this"])

[(0, 0)]

## Zad 2

In [7]:
for ascii_code in range(ord('a'), ord('z')+1):
    letter = chr(ascii_code)
    result = find_2d_pattern(lines, [letter, letter])
    if result != []:
        print(f'Letter "{letter}" on two neighbouring lines found on positions:\n{result}')
    

Letter "a" on two neighbouring lines found on positions:
[(64, 2), (37, 4), (20, 6), (56, 11), (52, 12), (53, 12), (64, 14), (76, 21), (64, 22), (59, 24), (3, 30), (65, 35), (69, 35), (57, 36), (58, 36), (79, 37), (77, 42), (53, 48), (31, 50), (78, 59), (5, 60), (77, 61), (6, 63), (33, 66), (28, 69), (31, 73), (76, 74), (0, 82)]
Letter "c" on two neighbouring lines found on positions:
[(41, 0), (68, 0), (13, 10), (82, 41), (10, 45), (3, 54)]
Letter "d" on two neighbouring lines found on positions:
[(37, 19)]
Letter "e" on two neighbouring lines found on positions:
[(10, 1), (14, 2), (24, 3), (17, 6), (76, 6), (77, 6), (80, 6), (1, 8), (20, 10), (40, 11), (81, 11), (81, 14), (69, 15), (67, 17), (72, 23), (40, 26), (18, 27), (73, 27), (51, 31), (42, 36), (29, 38), (71, 38), (15, 43), (29, 43), (68, 46), (82, 47), (37, 48), (42, 48), (70, 49), (47, 50), (58, 50), (46, 52), (22, 53), (57, 54), (58, 54), (41, 57), (21, 61), (0, 63), (10, 64), (7, 65), (24, 65), (78, 65), (63, 66), (28, 67),

## Zad 3

In [8]:
th, th_w_i_d_e = ["th", "th"], ["t h", "t h"]
th_positions = find_2d_pattern(lines, th)
th_w_i_d_e_positions = find_2d_pattern(lines, th_w_i_d_e)

print(f'{th} found on positions:\n{th_positions}')
print(f'{th_w_i_d_e} found on positions:\n{th_w_i_d_e_positions}')

['th', 'th'] found on positions:
[]
['t h', 't h'] found on positions:
[(27, 0), (37, 0)]


## Zad 4
**Wczytanie i przetworzenie obrazka i liter**

In [15]:
haystack_matrix = load_image("haystack.png")

pattern_t = load_image("patterns/letter_t.png")
pattern_h = load_image("patterns/letter_h.png")
pattern_e = load_image("patterns/letter_e.png")

print(pattern_e)

[[1 1 1 1 1 1 1 1 1 1 1]
 [1 1 1 1 0 0 0 0 1 1 1]
 [1 1 0 0 0 0 0 0 0 1 1]
 [1 1 0 0 1 1 1 1 0 0 1]
 [1 0 0 1 1 1 1 1 0 0 1]
 [1 0 0 0 0 0 0 0 0 0 1]
 [1 0 0 0 0 0 0 0 0 0 1]
 [1 0 0 1 1 1 1 1 1 1 1]
 [1 1 0 0 1 1 1 1 1 0 1]
 [1 1 0 0 0 0 0 0 0 0 1]
 [1 1 1 1 0 0 0 0 1 1 1]
 [1 1 1 1 1 1 1 1 1 1 1]]


**Wyszukiwanie wzorca**

In [16]:
matches_t = find_2d_pattern(haystack_matrix, pattern_t)
matches_h = find_2d_pattern(haystack_matrix, pattern_h)
matches_e = find_2d_pattern(haystack_matrix, pattern_e)

print(f'Matches for t: {matches_t}')
print(f'Matches for h: {matches_h}')
print(f'Matches for e: {matches_e}')

Matches for t: [(1772, 315)]
Matches for h: [(494, 32), (1440, 35), (1704, 36), (164, 38), (648, 38), (846, 38), (868, 38), (318, 42), (626, 42), (1418, 42), (978, 43), (736, 54), (406, 55), (692, 55), (1110, 55), (1616, 55), (296, 57), (824, 57), (1792, 57), (1418, 69), (1704, 71), (956, 80), (1484, 80), (296, 92), (1462, 92), (648, 96), (32, 97), (1770, 97), (1836, 104), (142, 108), (274, 108), (538, 109), (582, 113), (714, 113), (428, 116), (1044, 119), (1748, 120), (626, 124), (1594, 129), (362, 136), (1638, 142), (1572, 150), (956, 151), (736, 160), (1594, 167), (1660, 174), (1792, 177), (758, 178), (164, 182), (1550, 182), (54, 185), (98, 186), (1506, 186), (1154, 195), (274, 200), (1462, 202), (538, 207), (890, 214), (1748, 219), (1792, 219), (120, 221), (1814, 227), (1638, 233), (1704, 238), (956, 240), (714, 243), (1198, 245), (1286, 245), (1220, 247), (868, 249), (736, 251), (1484, 254), (384, 255), (934, 256), (1352, 265), (230, 266), (1132, 277), (1484, 278), (1704, 286), (

## Zad 5

In [18]:
p_a_t_t_e_r_n_pattern = load_image("patterns/pattern.png")
matches_p_a_t_t_e_r_n = find_2d_pattern(haystack_matrix, p_a_t_t_e_r_n_pattern)
print(matches_p_a_t_t_e_r_n)

[]


Najwidoczniej literka *t* oraz wzór *p a t t e r n* są źle wycięte. Niestety, pomimo wielu starań, nie udało mi się wyciąć tych plików w odpowiedni sposób.

## Zad 6

In [19]:
from time import time

def benchmark(file_name):
    file_pattern = load_image(file_name)
    a = time()
    automata = AhoCorasickAutomata(file_pattern)
    b = time()
    print(f"Time of preparing the automata: {round(b-a, 4)} [s]")
    a = time()
    find_2d_pattern(haystack_matrix, file_pattern)
    b = time()
    print(f"Time of finding the pattern: {round(b-a, 4)} [s]")
    
print("Small file:")
benchmark("patterns/small.png")
print()

print("Medium file:")
benchmark("patterns/medium.png")
print()

print("Large file:")
benchmark("patterns/large.png")
print()

Small file:
Time of preparing the automata: 0.0013 [s]
Time of finding the pattern: 1.1978 [s]

Medium file:
Time of preparing the automata: 0.025 [s]
Time of finding the pattern: 1.2876 [s]

Large file:
Time of preparing the automata: 3.9654 [s]
Time of finding the pattern: 6.3337 [s]



## Zad 7

In [ ]:
def chop_file(matrix, no_chops):
    size_of_one_chunk = matrix.shape[0]//no_chops
    return [matrix[i*size_of_one_chunk:(i+1)*size_of_one_chunk] for i in range(no_chops)]

def benchmark_chunks(aho_corasick, no_chunks):
    chopped_n_times = chop_file(haystack_matrix, no_chunks)
    a = time()
    for chunk in chopped_n_times:
        find_2d_pattern(chunk, p_a_t_t_e_r_n_pattern, aho_corasick)
    b = time()
    print(f"{no_chunks} chunks: {round(b-a, 4)} [s]")
    
aho_corasick = AhoCorasickAutomata(p_a_t_t_e_r_n_pattern)
for i in [2,4,8]:
    benchmark_chunks(aho_corasick, i)